<a href="https://colab.research.google.com/github/pamsfih/descomplicafood/blob/main/Agente_para_intolerantes_a_lactose_e_gluten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ[""] = userdata.get('') ## inclua sua chave da API

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [77]:
##########################################
# --- Agente 1: Receptor e Buscador de conteúdos sobre intolerância a lactose, doença celíaca e intolerância a glúten --- #
##########################################
def agente_receptor_buscador(assunto):

    receptor_buscador = Agent(
        name="agente_receptor_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um guia alimentar especializado amigável em intolerância a lactose, doença celíaca e intolerância a glúten. Muitas pessoas que possuem esses tipos de restrição alimentar acabam se sentindo excluídas e tendo dificuldades
        para encontrar informações confiáveis sobre dicas para lidar melhor com isso, receitas que podem ser feitas sem glúten e lactose, como se adaptar quando precisa sair ou viajar, entender o que são essas condições
        e impactos e também alimentos que podem substituir. Você não substitui um médico ou nutricionista, no entanto, pode apoiar esses profissionais e as pessoas que possuem essas condições. Não precisa de saudação, vá direto para a resposta.

        Suas tarefas são:

        - Usar a ferramenta de busca do google (google_search) para encontrar conteúdos em artigos, blogs e outros meios que seja fontes idôneas e confiáveis.
        - Priorizar sites que dizem respeito a área da saúde para auxiliar na confiabilidade do que será passado para as pessoas.
        - Focar em no máximo 3 conteúdos sobre o assunto para não ficar extenso.
        - Formatar a resposta de acordo com os temas de maneira organizada e com espaçamentos.
        - Inclua links úteis (receitas, artigos, vídeos) se encontrar, formatados como: `[Título do Link](URL do Link)`
        - Garanta que os links realmente funcionam.
        - Oferecer dicas práticas para o dia a dia, como ler rótulos, evitar contaminação cruzada, viajar com restrições alimentares.

        **Exemplo de Formato:**

        `<INFO_FUNDAMENTAL>
        O glúten é uma proteína encontrada no trigo...
        A lactose é um tipo de açúcar encontrado no leite...
        Sintomas: inchaço, dor abdominal... (Lembre-se: NÃO faça diagnóstico!)
        Rótulos: procure por 'glúten', 'lactose', 'caseína'...

        **Para saber mais, é só acessar os links:**

        - [Receitas sem Glúten](https://receitas.com/sem-gluten)
        - [Vídeo sobre Lactose](http://youtube.com/lactose)
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Assunto: {assunto}"

    informacoes = call_agent(receptor_buscador, entrada_do_agente_buscador)
    return informacoes

In [81]:
################################################
# --- Agente 2: Exibidor das informações encontradas --- #
################################################
def agente_exibidor(assunto, informacoes):
    exibidor = Agent(
        name="agente_exibidor",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Exibidor #################################################
        instruction="""
        Você é um apresentador de informações e também especialista em linguagem simples para a área da saúde e sua função é traduzir o que for técnico demais em informação amigável para as pessoas. Você é empático
        e sabe que as pessoas com restrições alimentares passam por algumas dificuldades na rotina, principalmente quem acabou de descobrir que possui essas restrições e não sabe como se adaptar. Você jamais substituirá
        um médico ou nutricionista, mas pode apoiar as pessoas e os próprios profissionais a garantir que as pessoas tenham acesso a informação e se adaptem melhor.

        - Utilize linguagem simples para falar com essas pessoas e tenha um tom orientativo, usando termos conhecidos.
        - Não seja formal, imagine que você conversa com pessoas de várias idades e leigas no assunto.
        - Não precisa de saudação, vá direto para a resposta.
        - Use poucos emojis, esse assunto é considerado mais sensível para algumas pessoas. Pense em acessibilidade jamais usando emojis como subsituto de bullet points,
        nem use mais de um emoji seguido.
        - Evite textos longos que cansam as pessoas, então seja objetivo, mas esteja próximo da pessoa.
        - Use Markdown para organizar o texto (títulos, listas).
        - Use HTML para criar links clicáveis.
        - Deixe a resposta visualmente amigável e fácil de ler, pense na hierarquia da informação.
        - Incentive o usuário a explorar os links e aprender mais!
        - De preferência, deixe os títulos em negrito e não use itálico.
        - Não use textos longos que dificultam o entendimento e acabam sendo cansativos, só responda o que é a pessoa perguntou.
        - Prioriza usar listas para facilitar a leitura.
        - Seja criativo e use uma linguagem que motive o usuário a aprender mais e a se sentir melhor!
        - Nos títulos comece com letra maiúscula e continue com minúscula.
        - Seja gentil e bem humorado, mas não faça piadas.

        Exemplo de resposta:

        ## O que é intolerância à lactose? 🥛

        A intolerância à lactose é a dificuldade em digerir a lactose, um tipo de açúcar encontrado no leite e em outros produtos lácteos.

        Exemplo de como usar maiúsculas e minúscula em lista:

        Como essa contaminação acontece?

        Direta: alimentos crus encostam em alimentos prontos para comer. Por exemplo, você corta um frango cru e, sem lavar a faca, corta uma abobrinha para salada.
        Indireta: acontece quando usamos utensílios, superfícies ou mãos sujas. Por exemplo, usamos a mesma tábua para cortar queijo e carne sem lavar entre os usos.

        ---------------

        Outro exemplo:

        Pesquise restaurantes: antes de sair, procure restaurantes que ofereçam opções sem lactose ou com alternativas para intolerantes.

        **Para saber mais, é só acessar os links:**

        * [Receitas sem glúten](https://www.exemplo.com/receitas-sem-gluten)
        * [Onde encontrar produtos sem lactose](https://www.exemplo.com/produtos-sem-lactose)
        """,

        description="Agente que exibe as respostas",
        tools=[google_search]
    )

    entrada_do_agente_exibidor = f"Assunto:{assunto}\nInformações: {informacoes}"

    resposta_exibida = call_agent(exibidor, entrada_do_agente_exibidor)
    return resposta_exibida

In [82]:
# Interagir com o bot

print("Iniciando o Descomplica Food: Guia de intolerância a lactose e glúten 🥬")
print ("""\nEu sou o Leo e estou aqui para te ajudar a lidar melhor com as restrições alimentares. Você pode ficar a vontade para fazer perguntas sobre:

- O que é glúten e lactose
- Sintomas comuns de intolerância
- Dicas práticas para identificar a presença de glúten e lactose em rótulos de alimentos, incluindo os termos mais comuns a serem observados
- Substituições e alternativas ao glúten e lactose
- Dicas de alimentação fora de casa

Ah, sempre consulte com uma pessoa nutricionista ou médica para ter um diagnóstico preciso e orientações para o seu caso específico. Cada organismo reage de uma maneira, mas é possível viver normal. 😊
"""),

# --- Obter o assunto do usuário ---
assunto = input("Me conta: o que você gostaria de saber? ")

# Lógica do sistema de agentes ######

if not assunto:
    print("Você esqueceu de digitar a sua dúvida!")
else:
    print(f"\nVamos falar sobre: {assunto}")

    informacoes = agente_receptor_buscador(assunto)
    print("\n--- 📝 Resultado do Agente 1 (Receptor e Buscador) ---\n")
    display(to_markdown(informacoes))
    print("--------------------------------------------------------------")

    resposta_exibida = agente_exibidor(assunto, informacoes)
    print("\n--- 📝 Resultado do Agente 2 (Exibidor) ---\n")
    display(to_markdown(resposta_exibida))
    print("--------------------------------------------------------------")

Iniciando o Descomplica Food: Guia de intolerância a lactose e glúten 🥬

Eu sou o Leo e estou aqui para te ajudar a lidar melhor com as restrições alimentares. Você pode ficar a vontade para fazer perguntas sobre:

- O que é glúten e lactose
- Sintomas comuns de intolerância
- Dicas práticas para identificar a presença de glúten e lactose em rótulos de alimentos, incluindo os termos mais comuns a serem observados
- Substituições e alternativas ao glúten e lactose
- Dicas de alimentação fora de casa

Ah, sempre consulte com uma pessoa nutricionista ou médica para ter um diagnóstico preciso e orientações para o seu caso específico. Cada organismo reage de uma maneira, mas é possível viver normal. 😊

Me conta: o que você gostaria de saber? o que é gluten

Vamos falar sobre: o que é gluten

--- 📝 Resultado do Agente 1 (Receptor e Buscador) ---



> O glúten é um conjunto de proteínas (gliadina e glutenina) encontrado em cereais como trigo, centeio, cevada e malte. Ele age como uma "cola", proporcionando elasticidade e viscosidade a massas e outros alimentos.
> 
> **Para entender melhor:**
> 
> *   O glúten é responsável pela textura macia e elástica de muitos alimentos à base de grãos.
> *   A maioria das pessoas pode consumir glúten sem problemas, mas algumas condições médicas exigem sua restrição.
> 
> **Alimentos que contêm glúten:**
> 
> *   Pães, bolos, biscoitos, massas, cervejas, entre outros.
> *   É importante ler os rótulos dos alimentos, pois o glúten pode estar presente em ingredientes como temperos industrializados, molhos e até alguns queijos.
> 
> **Impactos e condições relacionadas ao glúten:**
> 
> *   **Doença Celíaca:** Reação autoimune grave que danifica o intestino delgado.
> *   **Sensibilidade ao Glúten Não Celíaca:** Causa sintomas semelhantes aos da doença celíaca, mas sem os mesmos danos intestinais.
> *   **Alergia ao Trigo:** Intolerância ao trigo, não necessariamente à proteína do glúten em si.
> 
> **Substituições e alternativas:**
> 
> *   Farinhas de arroz, batata, soja, amaranto, quinoa, trigo sarraceno, feijão branco, farinha de milho, polvilho doce e azedo e tapioca são opções sem glúten para substituir a farinha de trigo.
> 
> **Dicas Importantes:**
> 
> *   **Rótulos:** Sempre verifique os rótulos dos alimentos para identificar a presença de glúten.
> *   **Contaminação Cruzada:** Tenha cuidado com a contaminação cruzada durante o preparo dos alimentos, especialmente se você tem doença celíaca ou sensibilidade ao glúten.
> *   **Aveia:** Nem toda aveia é livre de glúten, certifique-se de que o produto seja certificado como tal.
> *   **Moderação:** Mesmo alimentos sem glúten podem ser calóricos, portanto, consuma-os com moderação.
> 
> **Links úteis:**
> 
> *   [Doença Celíaca e Dietas sem glúten](https://www.nutrimento.pt/noticias/doenca-celiaca-e-dietas-sem-gluten/)
> *   [Dieta Isenta de glúten](https://apceliacos.pt/dieta-isenta-de-gluten/)
> *   [Receitas sem Glúten](https://www.nestle.pt/receitas/sem-gluten)
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Exibidor) ---



> ## Entendendo o que é glúten 🤔
> 
> O glúten é um grupo de proteínas, como a gliadina e a glutenina, encontrado em grãos como trigo, centeio, cevada e malte. Ele funciona como uma "cola", dando elasticidade e viscosidade a massas e outros alimentos.
> 
> *   É ele que deixa pães e bolos fofinhos e elásticos!
> *   A maioria das pessoas não tem problemas em consumir glúten, mas algumas precisam evitar por causa de condições médicas.
> 
> ### Onde o glúten se esconde? 🌾
> 
> *   Pães, bolos, biscoitos, massas e cervejas são alguns exemplos.
> *   Fique de olho nos rótulos! O glúten pode estar em temperos, molhos e até em alguns queijos.
> 
> ### O glúten pode ser um problema? ⚠️
> 
> *   **Doença Celíaca:** É uma reação autoimune que prejudica o intestino delgado.
> *   **Sensibilidade ao Glúten Não Celíaca:** Causa sintomas parecidos com a doença celíaca, mas sem danificar o intestino.
> *   **Alergia ao Trigo:** É uma intolerância ao trigo, e nem sempre é causada pelo glúten em si.
> 
> ### Como substituir o glúten? 🔄
> 
> *   Farinhas de arroz, batata, soja, amaranto, quinoa, trigo sarraceno, feijão branco, farinha de milho, polvilho doce e azedo e tapioca são ótimas opções para substituir a farinha de trigo.
> 
> ### Dicas importantes para o dia a dia 💡
> 
> *   **Sempre leia os rótulos** dos alimentos para verificar se contêm glúten.
> *   **Cuidado com a contaminação cruzada** ao preparar alimentos, especialmente se você tem doença celíaca ou sensibilidade ao glúten.
> *   **Verifique se a aveia é certificada** como livre de glúten.
> *   **Moderação é a chave!** Mesmo os alimentos sem glúten podem ter muitas calorias.
> 
> **Para saber mais, acesse os links:**
> 
> *   [Doença Celíaca e Dietas sem glúten](https://www.nutrimento.pt/noticias/doenca-celiaca-e-dietas-sem-gluten/)
> *   [Dieta Isenta de glúten](https://apceliacos.pt/dieta-isenta-de-gluten/)
> *   [Receitas sem Glúten](https://www.nestle.pt/receitas/sem-gluten)
> 


--------------------------------------------------------------
